<a href="https://colab.research.google.com/github/Scaraptor/Scaraptor/blob/main/kaggle_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [259]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [260]:
df = pd.read_csv("/content/drive/MyDrive/Data For ML/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Data For ML/test.csv")
# checker = pd.read_csv("/content/drive/MyDrive/Data For ML/sample_submission.csv")

In [261]:
##This is done to remove columns where too many values are missing(but on hindsight it reducing accuracy so i removed this step)
# for column in df:
#     if (df[column].isnull().sum())>700:
#         df.drop([column],axis = 1)

In [262]:
##If this is not done sklearn will throw error because some objects are present in training but not testing
##Collecting all the objects(strings)
object_cols = [col for col in df.columns if df[col].dtype == 'object']
##Checking if they are present in both df and df_test
good_label_cols = [col for col in object_cols if set(df_test[col]).issubset(set(df[col]))]

bad_label_cols = list(set(object_cols)-set(good_label_cols))
print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be ordinal encoded: ['Street', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['KitchenQual', 'Functional', 'SaleType', 'MSZoning', 'Exterior1st', 'Exterior2nd', 'Utilities']


In [263]:
##Ordinal Encoder changes all string(objects) in the columns to integers
ordinal_encoder = OrdinalEncoder()
df = df.drop(bad_label_cols, axis = 1)
df_test = df_test.drop(bad_label_cols, axis = 1)
df[good_label_cols] = ordinal_encoder.fit_transform(df[good_label_cols])
df_test[good_label_cols] =  ordinal_encoder.transform(df_test[good_label_cols])

In [264]:
Y = df['SalePrice']
df = df.drop(['SalePrice'],axis = 1)

In [265]:
Y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [266]:
##Simple Imputer removes all the NaN values are replaces it appropriately 
from sklearn.impute import SimpleImputer


my_imputer = SimpleImputer()
df = pd.DataFrame(my_imputer.fit_transform(df))
df_test = pd.DataFrame(my_imputer.transform(df_test))


In [267]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
0,1.0,60.0,65.0,8450.0,1.0,0.450549,3.0,3.0,4.0,0.0,5.0,2.0,2.0,0.0,5.0,7.0,5.0,2003.0,2003.0,1.0,1.0,1.0,196.0,2.0,4.0,2.0,2.0,3.0,3.0,2.0,706.0,5.0,0.0,150.0,856.0,1.0,0.0,1.0,4.0,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,8.0,0.0,2.733766,1.0,2003.0,1.0,2.0,548.0,4.0,4.0,2.0,0.0,61.0,0.0,0.0,0.0,0.0,1.142857,1.427046,1.907407,0.0,2.0,2008.0,4.0
1,2.0,20.0,80.0,9600.0,1.0,0.450549,3.0,3.0,2.0,0.0,24.0,1.0,2.0,0.0,2.0,6.0,8.0,1976.0,1976.0,1.0,1.0,2.0,0.0,3.0,4.0,1.0,2.0,3.0,1.0,0.0,978.0,5.0,0.0,284.0,1262.0,1.0,0.0,1.0,4.0,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,6.0,1.0,4.000000,1.0,1976.0,1.0,2.0,460.0,4.0,4.0,2.0,298.0,0.0,0.0,0.0,0.0,0.0,1.142857,1.427046,1.907407,0.0,5.0,2007.0,4.0
2,3.0,60.0,68.0,11250.0,1.0,0.450549,0.0,3.0,4.0,0.0,5.0,2.0,2.0,0.0,5.0,7.0,5.0,2001.0,2002.0,1.0,1.0,1.0,162.0,2.0,4.0,2.0,2.0,3.0,2.0,2.0,486.0,5.0,0.0,434.0,920.0,1.0,0.0,1.0,4.0,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,6.0,1.0,4.000000,1.0,2001.0,1.0,2.0,608.0,4.0,4.0,2.0,0.0,42.0,0.0,0.0,0.0,0.0,1.142857,1.427046,1.907407,0.0,9.0,2008.0,4.0
3,4.0,70.0,60.0,9550.0,1.0,0.450549,0.0,3.0,0.0,0.0,6.0,2.0,2.0,0.0,5.0,7.0,5.0,1915.0,1970.0,1.0,1.0,2.0,0.0,3.0,4.0,0.0,3.0,1.0,3.0,0.0,216.0,5.0,0.0,540.0,756.0,1.0,2.0,1.0,4.0,961.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,7.0,1.0,2.000000,5.0,1998.0,2.0,3.0,642.0,4.0,4.0,2.0,0.0,35.0,272.0,0.0,0.0,0.0,1.142857,1.427046,1.907407,0.0,2.0,2006.0,0.0
4,5.0,60.0,84.0,14260.0,1.0,0.450549,0.0,3.0,2.0,0.0,15.0,2.0,2.0,0.0,5.0,8.0,5.0,2000.0,2000.0,1.0,1.0,1.0,350.0,2.0,4.0,2.0,2.0,3.0,0.0,2.0,655.0,5.0,0.0,490.0,1145.0,1.0,0.0,1.0,4.0,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,9.0,1.0,4.000000,1.0,2000.0,1.0,3.0,836.0,4.0,4.0,2.0,192.0,84.0,0.0,0.0,0.0,0.0,1.142857,1.427046,1.907407,0.0,12.0,2008.0,4.0


In [268]:
X = df

In [269]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2)

In [270]:
clf = LinearRegression()
clf.fit(X_test,Y_test)
accuracy = clf.score(X_test, Y_test)
print(accuracy)
pred = clf.predict(df_test)


0.9372978747751158


In [271]:
print(pred)

[109798.40905306 173653.59842874 180953.72337487 ... 148414.05438766
  96867.89701432 216934.4406769 ]


In [272]:
# checker.drop(['Id'], axis = 1, inplace = True)

In [273]:
# checker.to_numpy()
# print(checker)
# print(mean_absolute_error(checker,pred))

In [275]:
pred = pd.DataFrame(pred)
sub_df = pd.read_csv("/content/drive/MyDrive/Data For ML/sample_submission.csv")
datasets = pd.concat([sub_df["Id"],pred],axis=1)
datasets.columns = ['Id','SalePrice']
datasets.to_csv('sample_submission.csv',index = False)